#  Entregable 3 - Aprendizaje Automático II


***<p style="text-align:center;">Implementación de una MLP con Backpropagation</p>***

En este cuaderno vamos a implementar una MLP (Multi-Layer Perceptron) con Backpropagation haciendo uso exclusivo de `numpy`.

### Normas de Entrega

1. El formato de entrega será en una carpeta comprimida con nombre: {Iniciales de Nombre y Apellidos}_E3.zip, en Aula Virtual en la fecha señalada en la plataforma y comunicada en clase previamente.
    * Por ejemplo: Iván Ramírez Díaz ==> `IRD_E3.zip`
2. El contenido de dicha carpeta será:
    * Obligatorio: Notebook relleno del Entregable 3.
    * Opcional: Memoria (pdf), en caso de necesitar dar alguna explicación.
3. Antes de la entrega, se debe comprobar que el código completo funciona.
4. La entrega es individual.

### Evaluación

La práctica entregable tiene un peso global de 1/4 puntos (los 4 entregables son el 10% de la nota final).

La práctica entregable se calificará sobre 10 puntos. Las puntuaciones son las siguientes:

- **[Ejercicio 1]** Implementa las clases: `Linear`, `Sigmoid` y `MSELoss`. (3 puntos)
- **[Ejercicio 2]** Implementa la clase: `SGD`y el modelo `MLP`. (3 puntos)
- **[Ejercicio 3]**  Entrena con los datos propuestos, una MLP en regresión. (2 puntos)
- **[Ejercicio 4 (para nota)]** Entrena el modelo anterior adaptándolo al `breast cancer dataset`que ya conoces usando `MSELoss`. (1 punto)
- **[Ejercicio 5 (para nota)]** Entrena el modelo anterior adaptándolo al `breast cancer dataset`que ya conoces implementando previamente una función de pérdida de clasificación, por ejemplo, la *binary cross entropy*  `BXELoss`. (1 punto)


## **[Ejercicio 1]**

Implementa las clases: `Linear`, `Sigmoid` y `MSELoss`.

### Definición de clase base

Como hemos visto en clase, todas y cada una de las funciones en un framework de diferenciación automática implementan tanto la función en sí como sus derivadas. De esta manera, se habilita el uso del algoritmo de retropropagación.

Toda clase que implemente una función en una red neuronal, tendrá dos métodos fundamentales:
* forward: consiste en la evaluación de dicha función $f(x)$ para una entrada $x$ dada.
* backward: consiste en la retropropagación de las derivadas/jacobianos desde la salida hasta los parámetros

De forma genérica, vamos a definir unos atributos y métodos mínimos para cada función *diferenciable*:
* Si la función no es paramétrica (función de activación, por ejemplo):

    * `init`: la clase tendrá `self.p = None` (no tiene parámetros)
    * `foward`:
      1. El método recibirá: $x$, que será la salida de la función anterior.
      2. El método tendrá: `self.f`, que será la evaluación de la función $f(x)$.
      3. El método devolverá: `self.f`.
    * `backward`:
        1. El método recibirá: `dL_df`, que será la derivada de la pérdida hasta la salida de la función $f$.
        2. El método tendrá: `self.df_dx`, que será la derivada de la función $f$ con respecto a la entrada $x$.
        3. El método devolverá: `dL_dx`, que será la aplicación de la regla de la cadena $\frac{\partial L}{\partial x} = \frac{\partial f}{\partial x} \frac{\partial L}{\partial f}$

* Si, además, la función depende de una serie de parámetros:

    * `init`:
      1. La clase definirá en `self.p` una lista de parámetros a utilizar.
      2. La clase tendrá `self.dL_dp` que será una lista de las posibles evaluaciones de los gradientes de los parámetros.
    * `backward`:
      1. Se calculará además `df_dp`, es decir, las derivadas de $f$ con respecto a cada uno de los parámetros $\frac{\partial f}{\partial p_i}$.
      2. La clase calculará `dL_dp` aplicando la regla de la cadena y añadirá la evaluación a la lista `self.dL_dp`.

### Capa Linear (afín):

Siguiendo las indicaciones anteriores, consideramos una capa lineal definida tal que:

$$
f(x) = {\bf{W}} {\bf{x}} + {\bf{b}}
$$

donde ${\bf{W}} \in \mathbb{R}^{m,n}, {\bf{x}} \in \mathbb{R}^{n}, {\bf{b}} \in \mathbb{R}^{m}, f \in \mathbb{R}^{m}$.

Las matrices jacobianas de interés son:

$$
\frac{\partial f}{\partial {\bf{x}}} = {\bf{W}}^T
$$

$$
\frac{\partial f}{\partial {\bf{b}}} = {\bf{I}}
$$

$$
\frac{\partial f}{\partial {\bf{W}}} = \begin{bmatrix}{\bf{x}} & \cdots & 0 \\
                                                        \vdots & \ddots & 0 \\
                                                        0 & 0 &{\bf{x}} \end{bmatrix} \in \mathbb{R}^{m\cdot n, m}
$$

Sobre esta última matriz jacobiana, conviene hacer un comentario de implementación. En la formulación propuesta arriba, se asume ${\bf{W}}$ como un vector $\mathbb{R}^{m\cdot n}$, aunque en realidad es una matriz de tamaño $m \times n$. De esta manera, generalizan las derivadas y se pueden ir aplicando los jacobianos siempre de la misma manera, es decir, multiplicando por la izquierda a los gradientes previos. Esto es, de forma genérica: $\frac{\partial L}{\partial x} = \frac{\partial f}{\partial x} \frac{\partial L}{\partial f}$.

Sin embargo, esta matriz tiene muchos elementos a 0, lo cual no es eficiente en términos de memoria y operaciones a realizar en la multiplicación matricial. Una alternativa habitual (con abuso de notación) es definir esta derivada (no el Jacobiano) como:

$$
\frac{\partial f}{\partial {\bf{W}}} = x^T
$$

y, en vez de multiplicar por la izquierda, hacerlo por la derecha. Es decir, de forma genérica: $\frac{\partial L}{\partial x} =  \frac{\partial L}{\partial f} \frac{\partial f}{\partial x}$.

La ventaja es doble: 1) eliminamos las componentes a 0 y las operaciones innecesarias y 2) el resultado es una matriz que coincide en dimensiones con las de ${\bf{W}}$, lo cual es muy conveniente para la actualización de parámetros.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
class Linear:
    def __init__(self, input_dim, output_dim, std=1):
        # Inicializamos pesos aleatorios y sesgo
        self.W = std*np.random.randn(output_dim, input_dim)
        self.b = std*np.random.randn(output_dim, 1)

        ########################### COMPLETAR ###########################
        self.p =  {'W': self.W, 'b': self.b}
        self.dL_dp = {'dL_dW': np.zeros_like(self.W), 'dL_db': np.zeros_like(self.b)}
        ######################### FIN COMPLETAR #########################
    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        ########################### COMPLETAR ###########################
        self.x = x

        return  self.W @ x +self.b # Producto matricial
        ######################### FIN COMPLETAR #########################

    def backward(self, dL_df):
        ########################### COMPLETAR ###########################

        # Derivadas con respecto a la entrada x
        dL_dx = self.W.T @ dL_df

        # Derivadas con respecto a los parámetros

        # Con respecto a W
        self.dL_dW = dL_df @ self.x.T
        #self.dL_dW = dL_df.reshape(-1, 1) @ self.x.reshape(1, -1)

        # Con respecto a b
        self.dL_db = dL_df

        # Se añaden a la lista de gradientes
        #(Yo hice un dict)
        self.dL_dp['dL_dW']=self.dL_dW
        self.dL_dp['dL_db']=self.dL_db

        return dL_dx

        ######################### FIN COMPLETAR #########################


### Capa de Activación (sigmoide):

Siguiendo las indicaciones anteriores, consideramos una capa de activación definida por una sigmoide:

$$
f(x) = \sigma({\bf{x}}) = \frac{1}{1 + e^{-{\bf{x}}}}
$$

donde ${\bf{x}} \in \mathbb{R}^{n}, f \in \mathbb{R}^{n}$.

La derivada con respecto a la entrada es:

$$
\frac{d f}{d {\bf{x}}} = \sigma({\bf{x}}) \cdot (1 - \sigma({\bf{x}})) = \begin{bmatrix}\sigma({\bf{x}_1}) \cdot (1 - \sigma({\bf{x}_1}))\\
                                                        \vdots  \\
                                                        \sigma({\bf{x}_n}) \cdot (1 - \sigma({\bf{x}_n})) \end{bmatrix} \in \mathbb{R}^{n}
$$
La matriz jacobiana:

$$
\frac{\partial f}{\partial {\bf{x}}} = diag(\sigma({\bf{x}}) \cdot (1 - \sigma({\bf{x}}))) = \begin{bmatrix}\sigma({\bf{x}_1}) \cdot (1 - \sigma({\bf{x}_1})) & \cdots & 0 \\
                                                        \vdots & \ddots & 0 \\
                                                        0 & 0 &\sigma({\bf{x}_n}) \cdot (1 - \sigma({\bf{x}_n})) \end{bmatrix} \in \mathbb{R}^{ n,n}
$$

Similar a lo que ocurría antes con el parámetro ${\bf{W}}$, la matriz Jacobiana de la activación es diagonal, teniendo de nuevo gran cantidad de ceros e incrementando la complejidad del número de operaciones. Por ello, se suele utilizar la derivada $\frac{d f}{d {\bf{x}}}$ aplicada como producto de Hadamard, es decir, punto a punto:  $\frac{\partial L}{\partial x} = \frac{d f}{d {\bf{x}}} \odot \frac{\partial L}{\partial f}$


In [ ]:
class Sigmoid:
    def __init__(self):
        ########################### COMPLETAR ###########################
        self.p =None
        ######################### FIN COMPLETAR #########################

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        ########################### COMPLETAR ###########################
        self.f = 1 / (1 + np.exp(-x))
        return self.f
        ######################### FIN COMPLETAR #########################

    def backward(self, dL_df):
        ########################### COMPLETAR ###########################
        df_dx = self.f* (1-self.f)

        # Derivada con respecto a la entrada
        self.dL_df = df_dx * dL_df
        return self.dL_df
        ######################### FIN COMPLETAR #########################


### Función de Pérdida (MSE):

De forma similar, implementamos la función de pérdida MSE, con una diferencia.

Las funciones de pérdida, suelen corresponder al último nodo del grafo de operaciones, por lo que debe iniciar la cadena de retropropagación al "llamar" al método `backward` para un modelo dado como argumento.

In [ ]:
class MSELoss:
    def __init__(self):
        ########################### COMPLETAR ###########################
        self.p = None
        ######################### FIN COMPLETAR #########################

    def __call__(self, y_pred, y_true):
        return self.forward(y_pred, y_true)

    def forward(self, y_pred, y_true):
        ########################### COMPLETAR ###########################
        self.y_pred = y_pred
        self.y_true = y_true
        self.f = np.mean((y_pred - y_true) ** 2)
        return self.f
        ######################### FIN COMPLETAR #########################

    def backward(self, model=None):
        ########################### COMPLETAR ###########################
        self.df_dx =  (2 * (self.y_pred - self.y_true))

        if model is not None:
            model.backward(self.df_dx)
        return self.df_dx
        ######################### FIN COMPLETAR #########################


## **[Ejercicio 2]**

Implementa la clase: `SGD`y el modelo `MLP`

### Modelo de perceptrón multi-capa (MLP):

En esta clase, se instancian en el `init`, las distintas capas a utilizar en el modelo, así como sus parámetros (dimensiones de entrada y salida).

Además, del `forward`y `backward`, es conveniente añadir un método para resetear (poner a cero o null) los gradientes. Para ello, se añade `zero_grads`.

In [ ]:
class MLP:
    def __init__(self, input_dim, output_dim):
        ########################### COMPLETAR ###########################
        F1 =  Linear(input_dim, 128)
        # Capa lineal con 128 neuronas de salida
        Act1 = Sigmoid() # Capa de activación
        F2 = Linear(128, 64) # Capa lineal con 64 neuronas de salida
        Act2 = Sigmoid() # Capa de activación
        F3 = Linear(64,output_dim) # Capa de salida
        self.layers = [F1, Act1, F2, Act2, F3]
        ######################### FIN COMPLETAR #########################

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        ########################### COMPLETAR ###########################
        # Forward de la red
        for layer in self.layers:
          x=layer.forward(x)

        return x
        ######################### FIN COMPLETAR #########################

    def backward(self, dL_df):
        ########################### COMPLETAR ###########################
        # Backward de la red
        for layer in reversed(self.layers):
          dL_df=layer.backward(dL_df)
        ######################### FIN COMPLETAR #########################

    def zero_grads(self):
        ########################### COMPLETAR ###########################
        # Para cada capa, resetea los gradientes
        for i in range(0, len(self.layers),2):
            layer = self.layers[i]
            layer.dL_dp['dL_dW']= np.zeros_like(layer.W)
            layer.dL_dp['dL_db']= np.zeros_like(layer.b)


        ######################### FIN COMPLETAR #########################


### Stocastic-Gradient Descent (Optimizador)

Implementamos un optimizador (SGD) para actualizar los valores de los parámetros.

Este optimizador, tendrá, al menos, un método llamado `step`.

In [ ]:
class SGD:
    def __init__(self, lr):
        self.lr = lr

    def step(self, model):
        ########################### COMPLETAR ###########################
        # Para cada capa con parámetros
        for i in range(0, len(model.layers),2):
            layer = model.layers[i]
            # Para cada conjunto de parámetros de la capa
            # Para cada gradediente (de cada ejemplo) de cada parámetr
            # Promedio  los gradientes
            # Actualización con descenso de gradiente
            model.layers[i].W -= self.lr * np.mean(layer.dL_dp['dL_dW'])
            model.layers[i].b -= self.lr * np.mean(layer.dL_dp['dL_db'])

        ######################### FIN COMPLETAR #########################


## **[Ejercicio 3]**

Entrena con los datos propuestos, una MLP en regresión.

Generamos unos datos a partir de la superficie siguiente:

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Crear un grid uniforme en el espacio de entrada
grid_size = 50  # Tamaño del grid para un muestreo más denso
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
X1, X2 = np.meshgrid(x1, x2)

# Calcular y_true para cada punto en el grid sin ruido
Y_org = np.sin(2 * np.pi * X1) + np.cos(2 * np.pi * X2)

# Crear la gráfica de superficie
fig = go.Figure(data=[go.Surface(
    x=X1,
    y=X2,
    z=Y_org,
    colorscale='Inferno'
)])

# Configuración del layout
fig.update_layout(
    scene=dict(
        xaxis_title='X1',
        yaxis_title='X2',
        aspectmode='cube',  # Asegura escalado igual en todos los ejes
        xaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Fondo gris claro
        yaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Fondo gris claro
        zaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True)   # Fondo gris claro
    ),
    width=800,  # Aumenta el tamaño de la figura
    height=800,  # Aumenta el tamaño de la figura
    margin=dict(l=0, r=0, b=0, t=50),  # Reduce los márgenes
    title="Superficie de la función original de la que se obtienen muestras ruidosas",  # Añade el título
    title_font=dict(size=20, family='Arial, sans-serif'),  # Personaliza la fuente del título
    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),  # Ángulo de cámara predeterminado para mejor visualización
)

# Mostrar la gráfica
fig.show()


Dado el modelo anterior, generamos unas muestras de entrenamiento ruidosas:

In [ ]:
np.random.seed(1)

# Parámetros de entrada y salida
input_dim = 2   # Dimensiones de entrada
output_dim = 1  # Dimensiones de salida

# Generación de datos de ejemplo
n_samples = 250
X_train = np.random.rand(n_samples, input_dim)  # Entradas aleatorias
y_train = np.sin(2 * np.pi * X_train[:,0:1]) + np.cos(2 * np.pi * X_train[:,1:]) + 0.1 * np.random.randn(n_samples, output_dim)  # Salidas deseadas

Entrena, con el `batch_size` que consideres, la `MLP` que has creado anteriormente.

In [ ]:
epochs = 100

########################### COMPLETAR ###########################
# Instancia el modelo, la función de pérdida y el optimizador
lr =0.1
batch_size =1

model = MLP(input_dim, output_dim)
loss_fn = MSELoss()
opt =SGD(lr)
######################### FIN COMPLETAR #########################

losses = []

for e in tqdm(range(epochs)): # Para cada época
    loss = []
    indices= np.random.permutation(n_samples)
    for b in range(n_samples//batch_size): # Para cada batch
        # Obten el batch para el SGD
        ########################### COMPLETAR ###########################
        indices_batch = indices[b * batch_size : (b + 1) * batch_size]
        x_train_batch = X_train[indices_batch, :]
        y_train_batch =y_train[indices_batch]
        ######################### FIN COMPLETAR #########################

        for i in range(len(x_train_batch)): # Para cada ejemplo del batch

            ########################### COMPLETAR ###########################
            # 1
            y_pred =  model.forward(x_train_batch[i].T)# Reshape de cada ejemplo para pasar por el modelo
            loss_value = loss_fn.forward(y_pred, y_train_batch[i].T)
            # Backward
            dL_df=loss_fn.backward()
            model.backward(dL_df)
            ######################### FIN COMPLETAR #########################
        # Descenso de gradiente (optimizador)
        opt.step(model)
        # Reset de los gradientes calculados
        model.zero_grads()
        ######################### FIN COMPLETAR #########################

        losses.append(np.mean(loss))


  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 128)

In [ ]:
# Visualizar la pérdida
plt.figure()
plt.plot(np.asarray(losses))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Training Loss')
plt.show()

In [ ]:
import plotly.graph_objs as go

# Lista de predicciones
predictions = []

# Prediciones de entrenamiento
for i in range(len(X_train)):
    y_pred = model(X_train[i].reshape(1, -1).T).flatten()[0]  # Ensure single value output
    predictions.append(y_pred)

# Plots
fig = go.Figure()

# Plot predicciones
fig.add_trace(go.Scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=predictions,
    mode='markers',
    marker=dict(color='red', size=8, symbol='circle', opacity=0.8),
    name='y_pred',
))

# Plot y_train
fig.add_trace(go.Scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=y_train.flatten(),
    mode='markers',
    marker=dict(color='blue', size=8, symbol='circle', opacity=0.8),
    name='y_true',
))

# Ajuste del layout
fig.update_layout(
    scene=dict(
        xaxis_title='X1',
        yaxis_title='X2',
        aspectmode='cube',  # Ensures equal scaling of all axes
        xaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Light gray background
        yaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Light gray background
        zaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True)   # Light gray background
    ),
    width=800,  # Increase the size of the figure
    height=800,  # Increase the size of the figure
    margin=dict(l=0, r=0, b=0, t=0),  # Reduce margins
    title="Predición de la MLP",  # Add title
    title_font=dict(size=20, family='Arial, sans-serif'),  # Customize title font
    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),  # Set the default camera angle for better visualization
)

fig.show()



## **[Ejercicio 4 (para nota)]**

Entrena el modelo anterior adaptándolo al `breast cancer dataset`que ya conoces usando `MSELoss`.


In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


########################### COMPLETAR ###########################
# Cargar el dataset
data= load_breast_cancer()
X=data.data
y=data.target

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


######################### FIN COMPLETAR #########################

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
# Añade un método `predict` para contemplar la predicción de varios ejemplos
class MLP:
    def __init__(self, input_dim, output_dim):
        ########################### COMPLETAR ###########################
        F1 =  Linear(input_dim, 128)
        # Capa lineal con 128 neuronas de salida
        Act1 = Sigmoid() # Capa de activación
        F2 = Linear(128, 64) # Capa lineal con 64 neuronas de salida
        Act2 = Sigmoid() # Capa de activación
        F3 = Linear(64,output_dim) # Capa de salida
        self.layers = [F1, Act1, F2, Act2, F3]

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        # Forward de la red
        for layer in self.layers:
          x=layer.forward(x)
        return x

    def backward(self, dL_df):        # Backward de la red
        for layer in reversed(self.layers):
          dL_df=layer.backward(dL_df)

    def zero_grads(self):
        # Para cada capa, resetea los gradientes
        for i in range(0, len(self.layers),2):
            layer = model.layers[i]
            layer.dL_dp['dL_dW']= np.zeros_like(layer.W)
            layer.dL_dp['dL_db']= np.zeros_like(layer.b)

    def predict(self, x):
        ########################### COMPLETAR ###########################

        # Para cada ejemplo del batch
        y_pred = self.forward(x.T)  # Asegúrate de que la entrada esté en el formato correcto
        return y_pred
        ######################### FIN COMPLETAR #########################


epochs = 500

########################### COMPLETAR ###########################
# Instancia el modelo, la función de pérdida y el optimizador
lr =0.1
batch_size =10

model =MLP(30,1)
loss_fn = MSELoss()
opt =SGD(lr)
######################### FIN COMPLETAR #########################
losses = []

for e in tqdm(range(epochs)): # Para cada época
    loss = []
    indices= np.random.permutation(n_samples)
    for b in range(n_samples//batch_size): # Para cada batch
        # Obten el batch para el SGD
        ########################### COMPLETAR ###########################
        indices_batch = indices[b * batch_size : (b + 1) * batch_size]
        x_train_batch = X_train[indices_batch, :]
        y_train_batch =y_train[indices_batch]
        ######################### FIN COMPLETAR #########################

        for i in range(len(x_train_batch)): # Para cada ejemplo del batch

            ########################### COMPLETAR ###########################
            # 1
            y_pred =  model.forward(x_train_batch[i].T)# Reshape de cada ejemplo para pasar por el modelo
            loss_value = loss_fn.forward(y_pred,y_train_batch[i])
            # Backward
            loss_fn.backward()
            ######################### FIN COMPLETAR #########################

            loss.append(loss_value)

        ########################### COMPLETAR ###########################
        # Descenso de gradiente (optimizador)
        opt.step(model)
        # Reset de los gradientes calculados
        model.zero_grads()
        ######################### FIN COMPLETAR #########################

        losses.append(np.mean(loss))

        ########################### COMPLETAR ###########################


        ######################### FIN COMPLETAR #########################

# Visualizar la pérdida
plt.plot(np.asarray(losses))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Pérdida de entrenamiento')
plt.show()

# Hacer predicciones en el conjunto de prueba
y_pred_test = model.predict(X_test)

y_pred_class = np.where(y_pred_test > 0.5, 1, 0)
# Calcular precisión en el conjunto de prueba
accuracy = np.mean(y_pred_class == y_test)
print(f"Precisión en el conjunto de test: {accuracy * 100:.2f}%")


## **[Ejercicio 5 (para nota)]**

Entrena el modelo anterior adaptándolo al `breast cancer dataset`que ya conoces implementando previamente una función de pérdida de clasificación, por ejemplo, la *binary cross entropy*  `BXELoss`.

In [ ]:
class BXELoss:
    def __init__(self):
        self.p = None

    def __call__(self, y_pred, y_true):
        return self.forward(y_pred, y_true)

    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true

        epsilon = 1e-12
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

        self.f = -np.mean(self.y_true * np.log(y_pred) + (1 - self.y_true) * np.log(1 - y_pred))
        return self.f

    def backward(self):
        epsilon = 1e-12
        y_pred = np.clip(self.y_pred, epsilon, 1 - epsilon)

        self.df_dx = -(self.y_true / y_pred - (1 - self.y_true) / (1 - y_pred))
        return self.df_dx

In [ ]:
# Añade un método `predict` para contemplar la predicción de varios ejemplos
class MLP:
    def __init__(self, input_dim, output_dim):
        ########################### COMPLETAR ###########################
        F1 =  Linear(input_dim, 128)
        # Capa lineal con 128 neuronas de salida
        Act1 = Sigmoid() # Capa de activación
        F2 = Linear(128, 64) # Capa lineal con 64 neuronas de salida
        Act2 = Sigmoid() # Capa de activación
        F3 = Linear(64,output_dim) # Capa de salida
        Act3=Sigmoid()
        self.layers = [F1, Act1, F2, Act2, F3, Act3]

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        # Forward de la red
        for layer in self.layers:
          x=layer.forward(x)
        return x

    def backward(self, dL_df):        # Backward de la red
        for layer in reversed(self.layers):
          dL_df=layer.backward(dL_df)

    def zero_grads(self):
        # Para cada capa, resetea los gradientes
        for i in range(0, len(self.layers),2):
            layer = model.layers[i]
            layer.dL_dp['dL_dW']= np.zeros_like(layer.W)
            layer.dL_dp['dL_db']= np.zeros_like(layer.b)

    def predict(self, x):
        ########################### COMPLETAR ###########################

        # Para cada ejemplo del batch
        y_pred = self.forward(x.T)
        y_pred_class = (y_pred > 0.5).astype(int)
        return y_pred
        ######################### FIN COMPLETAR #########################
